# 实战-天池资金流入流出预测(规则法)

## 导入数据

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [ ]:
###### Load the balance data
def load_data(path: str = '../PurchaseRedemptionData/user_balance_table.csv')->pd.DataFrame:
    data_balance = pd.read_csv(path)
    data_balance = add_timestamp(data_balance)
    return data_balance.reset_index(drop=True)
    

# add tiemstamp to dataset
def add_timestamp(data: pd.DataFrame, time_index: str = 'report_date')->pd.DataFrame:
    data_balance = data.copy()
    data_balance['date'] = pd.to_datetime(data_balance[time_index], format= "%Y%m%d")
    data_balance['day'] = data_balance['date'].dt.day
    data_balance['month'] = data_balance['date'].dt.month
    data_balance['year'] = data_balance['date'].dt.year
    data_balance['week'] = data_balance['date'].dt.week
    data_balance['weekday'] = data_balance['date'].dt.weekday
    return data_balance.reset_index(drop=True)

# total amount
def get_total_balance(data: pd.DataFrame, date: str = '2014-03-31')->pd.DataFrame:
    df_tmp = data.copy()
    df_tmp = df_tmp.groupby(['date'])['total_purchase_amt','total_redeem_amt'].sum()
    df_tmp.reset_index(inplace=True)
    return df_tmp[(df_tmp['date']>= date)].reset_index(drop=True)

# Generate the test data
def generate_test_data(data: pd.DataFrame)->pd.DataFrame:
    total_balance = data.copy()
    start = datetime.datetime(2014,9,1)
    testdata = []
    while start != datetime.datetime(2014,10,15):
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        start += datetime.timedelta(days = 1)
    testdata = pd.DataFrame(testdata)
    testdata.columns = total_balance.columns

    total_balance = pd.concat([total_balance, testdata], axis = 0)
    total_balance = total_balance.reset_index(drop=True)
    return total_balance.reset_index(drop=True)

# Load user's information
def load_user_information(path: str = '../PurchaseRedemptionData/user_profile_table.csv')->pd.DataFrame:
    return pd.read_csv(path)

In [ ]:
balance_data = load_data('./PurchaseRedemptionData/user_balance_table.csv')
balance_data = add_timestamp(balance_data)
total_balance = get_total_balance(balance_data, date = '2014-03-01')
total_balance = generate_test_data(total_balance)
total_balance = add_timestamp(total_balance, 'date')

In [ ]:
pred = total_balance[(total_balance['date'].dt.date > datetime.date(2014,8,31)) & (total_balance['date'].dt.date <= datetime.date(2014,9,30))]
pred['weekday'] = pred['date'].dt.weekday + 1
pred.head()

In [ ]:
df_temp = total_balance.copy()
df_temp_train = df_temp[(df_temp['date'].dt.date >= datetime.date(2014,3,31)) & (df_temp['date'].dt.date <= datetime.date(2014,8,31))]
# df_temp_test = df_temp[(df_temp['date'].dt.date > datetime.date(2014,8,3)) & (df_temp['date'].dt.date <= datetime.date(2014,8,31))]   

In [ ]:
df_temp_train.head(10)

In [ ]:
df_temp_train.reset_index(inplace=True)
# df_temp_test.reset_index(inplace=True)

In [ ]:
del df_temp_train['index']
# del df_temp_test['index']

In [ ]:
pred = pd.DataFrame(columns = ['report_date','purchase','redeem'])
pred['report_date'] = ["2014-08-04","2014-08-05","2014-08-06","2014-08-07","2014-08-08","2014-08-09","2014-08-10","2014-08-11","2014-08-12",
                      "2014-08-13","2014-08-14","2014-08-15","2014-08-16","2014-08-17","2014-08-18","2014-08-19","2014-08-20","2014-08-21",
                       "2014-08-22","2014-08-23","2014-08-24","2014-08-25","2014-08-26","2014-08-27","2014-08-28","2014-08-29","2014-08-30","2014-08-31"]
pred['purchase'] = df_temp_test['total_purchase_amt']
pred['redeem'] = df_temp_test['total_redeem_amt']
pred['report_date'] = pd.to_datetime(pred['report_date'])
pred['weekday'] = pred['report_date'].dt.weekday + 1

In [ ]:
df_purchase = pd.DataFrame(columns=['weekday1','weekday2','weekday3','weekday4','weekday5','weekday6','weekday7'])

count = 0
for i in range(len(df_temp_train)):
    df_purchase.loc[count,'weekday'+str(df_temp_train.loc[i,'weekday']+1)] = df_temp_train.loc[i,'total_purchase_amt']
    if df_temp_train.loc[i,'weekday'] == 6:
        count = count + 1

In [ ]:
df_purchase.head()

In [ ]:
df_redeem = pd.DataFrame(columns=['weekday1','weekday2','weekday3','weekday4','weekday5','weekday6','weekday7'])

count = 0
for i in range(len(df_temp_train)):
    df_redeem.loc[count,'weekday'+str(df_temp_train.loc[i,'weekday']+1)] = df_temp_train.loc[i,'total_redeem_amt']
    if df_temp_train.loc[i,'weekday'] == 6:
        count = count + 1

In [ ]:
df_redeem.head()

## 中位数

In [ ]:
df_purchase_median = df_purchase.median()
df_redeem_median = df_redeem.median()

for i in range(7):
    pred.loc[pred['weekday']==i+1,'purchaseMedian'] = df_purchase_median[i]

for i in range(7):
    pred.loc[pred['weekday']==i+1,'redeemMedian'] = df_redeem_median[i]

In [ ]:
pred.head()

## 周期因子

In [ ]:
# 第一种计算方式
def predictByWeekdayFactor(df_,weekId):
    df_['week_mean'] = df_.mean(axis=1)
    
    for i in range(7):
        df_['weekday'+str(i+1)+'_coefficient'] = df_['weekday'+str(i+1)]/df_['week_mean']
    
    coefficient_list = ['weekday1_coefficient','weekday2_coefficient','weekday3_coefficient','weekday4_coefficient','weekday5_coefficient','weekday6_coefficient','weekday7_coefficient']
    
    weekday_rate = df_[coefficient_list].median()
    
    base = np.array(df_.loc[weekId,['weekday1','weekday2','weekday3','weekday4','weekday5','weekday6','weekday7']]) * 1 / np.array(df_[coefficient_list].median())
            
    Pre = np.mean(base) * np.array([weekday_rate])
                     
    return Pre

In [ ]:
trade_weekday = df_temp_train.groupby(['weekday'])['total_purchase_amt','total_redeem_amt'].mean()
trade_weekday.reset_index(inplace=True)

purchase_mean = np.mean(df_temp_train['total_purchase_amt'])
redeem_mean = np.mean(df_temp_train['total_redeem_amt'])

purchase_weekday_rate_2 = trade_weekday['total_purchase_amt']/purchase_mean
redeem_weekday_rate_2 = trade_weekday['total_redeem_amt']/redeem_mean

basePurchase_2 = np.mean(df_purchase.loc[len(df_purchase)-1,['weekday1','weekday2','weekday3',
                                                           'weekday4','weekday5','weekday6','weekday7']]*1/np.array(purchase_weekday_rate_2))
purchasePre_2 = basePurchase_2 * purchase_weekday_rate_2

baseRedeem_2 = np.mean(df_redeem.loc[len(df_redeem)-1,['weekday1','weekday2','weekday3',
                                                           'weekday4','weekday5','weekday6','weekday7']]*1/np.array(redeem_weekday_rate_2))
redeemPre_2 = baseRedeem_2 * redeem_weekday_rate_2

In [ ]:
def predictByMonthDayFactor(df_,purchase_weekday_rate,redeem_weekday_rate):
    trade_day = df_.groupby(['day'])['total_purchase_amt','total_redeem_amt'].mean()
    trade_day.reset_index(inplace=True)
    
    for i in range(7):
        trade_day['weekday'+str(i+1)] = 0
        
    trade_day['purchase_day_rate'] = 0
    trade_day['redeem_day_rate'] = 0
    
    # 求1号至30号处于星期1到星期日的频次
    for i in range(31):
        for j in range(7):
            trade_day.loc[i,'weekday'+str(j+1)] = sum((df_['day']==i+1)&(df_['weekday']==j+1))
    
    for i in range(31):
        trade_day.loc[i,'purchase_day_rate'] = sum(np.array(purchase_weekday_rate)*np.array(trade_day.loc[i,['weekday1','weekday2','weekday3',
                                                           'weekday4','weekday5','weekday6','weekday7']]))/sum(trade_day.loc[i,['weekday1','weekday2','weekday3',
                                                           'weekday4','weekday5','weekday6','weekday7']])
        trade_day.loc[i,'redeem_day_rate'] = sum(np.array(redeem_weekday_rate)*np.array(trade_day.loc[i,['weekday1','weekday2','weekday3',
                                                           'weekday4','weekday5','weekday6','weekday7']]))/sum(trade_day.loc[i,['weekday1','weekday2','weekday3',
                                                           'weekday4','weekday5','weekday6','weekday7']])
        
    trade_day['purchasePre'] = trade_day['total_purchase_amt']/trade_day['purchase_day_rate'] # 均值除以加权后的周期因子,得到去周期化的均值
    trade_day['redeemPre'] = trade_day['total_redeem_amt']/trade_day['redeem_day_rate']
    
    return trade_day

trade_day = predictByMonthDayFactor(df_temp_train,purchase_weekday_rate_2,redeem_weekday_rate_2)

pred['day'] = pred['date'].dt.day
pred = pd.merge(pred,trade_day[['day','purchasePre','redeemPre']])

for i in range(len(pred)):
    pred.loc[i,'purchasePre'] = pred.loc[i,'purchasePre'] * purchase_weekday_rate_2[pred.loc[i,'weekday']-1]
    pred.loc[i,'redeemPre'] = pred.loc[i,'redeemPre'] * redeem_weekday_rate_2[pred.loc[i,'weekday']-1]

In [ ]:
pred['purchaseMedian_MAPE'] = np.abs(pred['purchaseMedian']-df_temp_test['total_purchase_amt'])/df_temp_test['total_purchase_amt']
pred['redeemMedian_MAPE'] = np.abs(pred['redeemMedian']-df_temp_test['total_redeem_amt'])/df_temp_test['total_redeem_amt']

h = 0.3
tmp = sum(pred['redeemMedian_MAPE'].apply(lambda x:np.exp(-x/h))*10) * 0.55 + sum(pred['purchaseMedian_MAPE'].apply(lambda x:np.exp(-x/h))*10) * 0.55

In [ ]:
for i in pred.columns:
    if i == 'date':
        pred[i] = pred[i].astype(str)
        pred[i] = pred[i].str.replace('-','')

In [ ]:
pred.head()

In [ ]:
# 保存预测结果到本地

data = pred[['date','purchaseMedian','redeemMedian']]
data.to_csv('../comp_predict_table.csv',index=False,header=None)